In [1]:
from mpdaf.obj import Cube
import numpy as np
import pandas as pd

In [2]:
from dfisher_2022a.emission_lines import Halpha
from dfisher_2022a.io.cube import FitReadyCube, RestCube, SNRMap
from dfisher_2022a.io.line import Line
import dfisher_2022a as dfi

### Read in test data cube

In [3]:
path = "../dfisher_2022a/tests/data/"
fits_file = path + "single_gaussian_muse_size.fits"
res_file = path + "eso338_test_run_6563_simple_model.txt"

In [4]:
cube = Cube(fits_file)

In [5]:
cube.shape

(34, 368, 331)

In [6]:
cube.wave.get_crval()

6603.8486328125

### Settings

In [7]:
Z = 0.009482649107040553
SNR_THRESHOLD = 5

### Prepare data for fitting

#### 1. de-redshift

In [8]:
rcube = RestCube(cube, Z)

In [9]:
rcube.z

0.009482649107040553

In [10]:
rcube._restwave

6541.814897615204

In [11]:
rcube.restcube.wave.get_crval()

6541.814897615204

2. #### select fitting region

In [12]:
fline = Line(Halpha)

In [13]:
print(fline.low, fline.line, fline.high)

6547.819 6562.819 6577.819


In [14]:
rcube.restcube.wave.pixel(6577.8199)

28.804001907823775

In [15]:
frcube = FitReadyCube(rcube, fline)

In [16]:
frcube.low_index

5

In [17]:
frcube.cube.shape

(25, 368, 331)

#### 3. get snr map

In [18]:
snrmap = SNRMap(rcube.restcube, SNR_THRESHOLD)

In [19]:
snr = snrmap.snr

In [20]:
snr[80,80]

masked

In [21]:
smap = snrmap.map

In [22]:
smap.data[80,80]

masked

In [23]:
snr.shape

(368, 331)

In [24]:
count = 0
for i in range(snr.shape[0]):
    for j in range(snr.shape[1]):
        if snr[i,j] >= 5:
            count += 1

In [25]:
count

69490

In [26]:
sm_cube = snrmap.snr_masked_cube

In [27]:
sm_cube.shape

(34, 368, 331)

In [28]:
te = sm_cube.data[:,80,80]

In [29]:
te.mask.all()

True

#### get fit ready cube

In [30]:
ffrcube = FitReadyCube(rcube, fline, SNR_THRESHOLD)

In [31]:
ffrcube.cube.data[:,150,150]

masked_array(data=[12.175702095031738, 11.767293930053711,
                   12.22632884979248, 11.773338317871094,
                   11.761051177978516, 11.782549858093262,
                   12.78128719329834, 11.9537935256958,
                   14.307246208190918, 30.050148010253906,
                   87.67877960205078, 187.4704132080078,
                   241.05686950683594, 179.9756317138672,
                   81.16941833496094, 27.804880142211914,
                   14.032210350036621, 11.993968963623047,
                   11.892915725708008, 12.09134578704834,
                   11.779748916625977, 12.0618257522583,
                   11.765948295593262, 11.759703636169434,
                   12.781892776489258],
             mask=[False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False],
       fill

### Fit the data

In [31]:
model = dfi.models.Lm_Const_1GaussModel

In [32]:
cube_for_fit = ffrcube.cube

In [33]:
fc = dfi.fits.base.FitCube(cube_for_fit, model)

/Users/SuperTiger/ADACS/repo/SS2022A-DFisher/dfisher_2022a/dfisher_2022a/fits/base.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["row"][i*axis_y:(i+1)*axis_y] = i


In [40]:
fc.res.dtypes

row       int32
col       int64
Result    int32
dtype: object

In [37]:
c = fc.fit_single_spaxel(150,150)

subprocess:  68493  pixel:  150 150


In [40]:
te = np.zeros((3,3))

In [41]:
te

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [34]:
small = cube_for_fit[:,140:150,140:150]

In [35]:
fsmall = dfi.fits.base.FitCube(small, model)

/Users/SuperTiger/ADACS/repo/SS2022A-DFisher/dfisher_2022a/dfisher_2022a/fits/base.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["row"][i*axis_y:(i+1)*axis_y] = i


In [36]:
fsmall.res

,row,col,Result
0,0,0,0
1,0,1,0
2,0,2,0
3,0,3,0
4,0,4,0
...,...,...,...
95,9,5,0
96,9,6,0
97,9,7,0
98,9,8,0


In [37]:
%%time

fsmall.fit_all(4)

/Users/SuperTiger/ADACS/repo/SS2022A-DFisher/dfisher_2022a/dfisher_2022a/fits/base.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.res["Result"].iloc[idx] = future.result()


subprocess:  70277  pixel:  0 0
subprocess:  70277  pixel:  0 4
subprocess:  70277  pixel:  0 8
subprocess:  70277  pixel:  1 2
subprocess:  70277  pixel:  1 6
subprocess:  70277  pixel:  2 0
subprocess:  70277  pixel:  2 4
subprocess:  70277  pixel:  2 8
subprocess:  70277  pixel:  3 2
subprocess:  70277  pixel:  3 6
subprocess:  70277  pixel:  4 0
subprocess:  70277  pixel:  4 4
subprocess:  70277  pixel:  4 8
subprocess:  70277  pixel:  5 2
subprocess:  70277  pixel:  5 6
subprocess:  70277  pixel:  6 0
subprocess:  70277  pixel:  6 4
subprocess:  70277  pixel:  6 8
subprocess:  70277  pixel:  7 2
subprocess:  70277  pixel:  7 6
subprocess:  70277  pixel:  8 0
subprocess:  70277  pixel:  8 4
subprocess:  70277  pixel:  8 8
subprocess:  70277  pixel:  9 2
subprocess:  70277  pixel:  9 6
subprocess:  70276  pixel:  0 1
subprocess:  70276  pixel:  0 5
subprocess:  70276  pixel:  0 9
subprocess:  70276  pixel:  1 3
subprocess:  70276  pixel:  1 7
subprocess:  70276  pixel:  2 1
subproce

In [39]:
fsmall.res.dtypes

row        int32
col        int64
Result    object
dtype: object

In [44]:
ch = fsmall.res["Result"][0]

In [45]:
dir(ch)

['_Minimizer__jacobian',
 '_Minimizer__residual',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abort',
 '_calculate_covariance_matrix',
 '_calculate_uncertainties_correlations',
 '_err_max_evals',
 '_err_nonparam',
 '_int2ext_cov_x',
 '_lnprob',
 '_repr_html_',
 'aborted',
 'aic',
 'ampgo',
 'basinhopping',
 'best_fit',
 'best_values',
 'bic',
 'brute',
 'calc_covar',
 'call_kws',
 'chisqr',
 'ci_out',
 'ci_report',
 'col_deriv',
 'components',
 'conf_interval',
 'covar',
 'data',
 'dual_annealing',
 'dump',
 'dumps',
 'emcee',
 'errorbars',
 'eval',
 'eval_components',
 'eval_uncertainty',
 'fit',
 'fit_report',
 'flatchain',
 'ier',
 'init_fit'

In [61]:
cr = ch.result

In [63]:
cr.var_names

['g1_height', 'g1_center', 'g1_sigma', 'c']

In [84]:
ch.params.get('g1_height').stderr

1.0535887534869381

In [78]:
dir(ch.params)

['__add__',
 '__array__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_asteval',
 '_repr_html_',
 'add',
 'add_many',
 'clear',
 'copy',
 'dump',
 'dumps',
 'eval',
 'fromkeys',
 'get',
 'items',
 'keys',
 'load',
 'loads',
 'pop',
 'popitem',
 'pretty_print',
 'pretty_repr',
 'setdefault',
 'update',
 'update_constraints',
 'values',
 'valuesdict']

In [56]:
ch.last_internal_values

array([1.38968256e+02, 6.56301940e+03, 2.21056375e+00, 5.27618346e+00])

In [58]:
ch.best_values

{'c': 5.2761834589929615,
 'g1_height': 137.97185427421581,
 'g1_center': 6563.019402970259,
 'g1_sigma': 1.4262300169236233}

In [85]:
type(ch)

lmfit.model.ModelResult

In [86]:
import sys

In [87]:
sys.getsizeof(ch)

48

In [88]:
id(ch)

5220214144

In [91]:
l = [ch,ch,ch]

In [94]:
sys.getsizeof(l)

86456

In [93]:
l = [ch]*10800

In [96]:
y = (1,2,'tr')

In [98]:
type(y)

tuple